In [149]:
import pandas as pd

# Load CSV files
match_results_df = pd.read_csv('csv/match_results.csv')
goal_events_df = pd.read_csv('csv/goal_events.csv')
standings_df = pd.read_csv('csv/standings.csv')

team_name_mapping = {
    'Antwerp FC': 'R Antwerp FC',
    'Cercle': 'Cercle Brugge',
    'Club Brugge': 'Club Brugge',
    'Eupen': 'KAS Eupen',
    'FCB': 'Club Brugge',
    'Genk': 'KRC Genk',
    'Gent': 'KAA Gent',
    'KAA Gent': 'KAA Gent',
    'KAS Eupen': 'KAS Eupen',
    'KV Kortrijk': 'KV Kortrijk',
    'KV Mechelen': 'KV Mechelen',
    'KVC Westerlo': 'KVC Westerlo',
    'KVCWes': 'KVC Westerlo',
    'KVK': 'KV Kortrijk',
    'KVM': 'KV Mechelen',
    'OH Leuven': 'Oud-Heverlee Leuven',
    'OHL': 'Oud-Heverlee Leuven',
    'R Charleroi SC': 'R Charleroi SC',
    'RAFC': 'R Antwerp FC',
    'RCSC': 'R Charleroi SC',
    'RSC Anderlecht': 'RSC Anderlecht',
    'RSCA': 'RSC Anderlecht',
    'RUSG': 'Union Saint-Gilloise',
    'RWDM': 'RWD Molenbeek',
    'SL': 'Standard Luik',
    'STVV': 'Sint-Truidense VV',
    'St-Truidense VV': 'Sint-Truidense VV',
    'Standard Luik': 'Standard Luik',
    'Union SG': 'Union Saint-Gilloise'
}

goal_events_df = goal_events_df[goal_events_df['valid_goal'] == True]

# Calculate goals for home and away teams
goals_home_team = goal_events_df[goal_events_df['goal_team'] == goal_events_df['home_team']].groupby('match_id').size()
goals_away_team = goal_events_df[goal_events_df['goal_team'] == goal_events_df['away_team']].groupby('match_id').size()

# Convert Series to DataFrame and reset index
goals_home_team_df = goals_home_team.reset_index(name='calculated_goals_home_team')
goals_away_team_df = goals_away_team.reset_index(name='calculated_goals_away_team')

# Replace NaN values in goal columns with 0
merged_df = pd.merge(match_results_df, goals_home_team_df, on='match_id', how='outer')
merged_df = pd.merge(merged_df, goals_away_team_df, on='match_id', how='outer')
merged_df['calculated_goals_home_team'] = merged_df['calculated_goals_home_team'].fillna(0)
merged_df['calculated_goals_away_team'] = merged_df['calculated_goals_away_team'].fillna(0)


# Calculate discrepancies in goals vs result
discrepancies = merged_df[
    (merged_df['calculated_goals_home_team'] != merged_df['result_home_team']) |
    (merged_df['calculated_goals_away_team'] != merged_df['result_away_team'])
]

In [150]:
# Select only relevant columns for display
discrepancies = discrepancies[['date', 'match_id', 'home_team', 'away_team', 'calculated_goals_home_team', 'result_home_team', 'calculated_goals_away_team', 'result_away_team']]
discrepancies.head(50)

,date,match_id,home_team,away_team,calculated_goals_home_team,result_home_team,calculated_goals_away_team,result_away_team
132,2007/12/08,79993,FC Brussel,KRC Genk,0.0,0,0.0,5
154,2008/01/19,80019,RSC Anderlecht,KV Mechelen,0.0,1,1.0,0


In [151]:
# controle van aantal_wedstrijden
# Geen enkel record met meer wedstrijden dan speeldagen
controle_aantal_wedstrijden = standings_df[standings_df.Day < standings_df.Played]
controle_aantal_wedstrijden.head(30)

,Rank,Club,Club Number,Played,Wins,Draws,Losses,Goals,Goal_Difference,Points,Season,Day


In [152]:
# controle van aantal_wedstrijden
# wel records met minder wedstrijden dan speeldagen, maar bij nazicht van een aantal records klopt dit
controle_aantal_wedstrijden = standings_df[standings_df.Day > standings_df.Played]
controle_aantal_wedstrijden.Season.value_counts()

Series([], Name: Season, dtype: int64)

In [153]:
# geen records met standings_df.Played != (standings_df.Wins + standings_df.Losses + standings_df.Draws
controle_aantal_wedstrijden = standings_df[standings_df.Played != (standings_df.Wins + standings_df.Losses + standings_df.Draws)]
controle_aantal_wedstrijden.count()

Rank               0
Club               0
Club Number        0
Played             0
Wins               0
Draws              0
Losses             0
Goals              0
Goal_Difference    0
Points             0
Season             0
Day                0
dtype: int64

In [154]:
# Split the 'Goals' column into two separate columns
standings_df[['Goals_Scored', 'Goals_Conceded']] = standings_df['Goals'].str.split(':', expand=True)

standings_df['Goals_Scored'] = pd.to_numeric(standings_df['Goals_Scored'])
standings_df['Goals_Conceded'] = pd.to_numeric(standings_df['Goals_Conceded'])

# geen records met (uitslagen.doelpunten_voor - uitslagen.doelpunten_tegen) != (uitslagen.verschil)
standings_df['Calculated_Goal_Difference'] = standings_df['Goals_Scored'] - standings_df['Goals_Conceded']
controle_aantal_wedstrijden = standings_df[standings_df['Calculated_Goal_Difference'] != standings_df['Goal_Difference']]
controle_aantal_wedstrijden.count()

Rank                          0
Club                          0
Club Number                   0
Played                        0
Wins                          0
Draws                         0
Losses                        0
Goals                         0
Goal_Difference               0
Points                        0
Season                        0
Day                           0
Goals_Scored                  0
Goals_Conceded                0
Calculated_Goal_Difference    0
dtype: int64

In [155]:
# lijnen zoeken waarvoor punten geen : bevat (seizoen 1964 + vanaf 1995)
controle_klassement = standings_df[(standings_df.Goals.str.count(':') == 0)]
controle_klassement.head()

,Rank,Club,Club Number,Played,Wins,Draws,Losses,Goals,Goal_Difference,Points,Season,Day,Goals_Scored,Goals_Conceded,Calculated_Goal_Difference


In [156]:
# het totaal aantal gewonnen matchen per seizoen en per ploeg
help = standings_df.groupby(['Season','Day','Club Number'])[['Wins','Draws','Losses']].sum()
help = help.reset_index()
help.head()

,Season,Day,Club Number,Wins,Draws,Losses
0,2007,1,12,0,0,1
1,2007,1,134,0,1,0
2,2007,1,16,1,0,0
3,2007,1,1611,0,0,1
4,2007,1,25,0,0,1


In [157]:
help = help.sort_values(['Season','Day','Wins','Draws','Losses'], ascending = [True, True, False, False, False])
help.head(30)

,Season,Day,Club Number,Wins,Draws,Losses
2,2007,1,16,1,0,0
5,2007,1,290,1,0,0
6,2007,1,3,1,0,0
7,2007,1,35,1,0,0
9,2007,1,7,1,0,0
10,2007,1,867,1,0,0
13,2007,1,Germ. Beerschot,1,0,0
14,2007,1,KSC Lokeren,1,0,0
1,2007,1,134,0,1,0
15,2007,1,R Charleroi SC,0,1,0


In [158]:
help['Points'] = help['Wins'] * 3 + help['Draws'] + help['Losses'] * 0
help.head(30)

,Season,Day,Club Number,Wins,Draws,Losses,Points
2,2007,1,16,1,0,0,3
5,2007,1,290,1,0,0,3
6,2007,1,3,1,0,0,3
7,2007,1,35,1,0,0,3
9,2007,1,7,1,0,0,3
10,2007,1,867,1,0,0,3
13,2007,1,Germ. Beerschot,1,0,0,3
14,2007,1,KSC Lokeren,1,0,0,3
1,2007,1,134,0,1,0,1
15,2007,1,R Charleroi SC,0,1,0,1
